In [8]:
import pandas as pd
import matplotlib as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import keras_tuner as kt


import tensorflow as tf
from sqlalchemy import create_engine


from sklearn.datasets import make_blobs, make_moons, make_circles
%matplotlib inline

In [9]:
from config import db_password, db_name, db_server

# Importing COVID Data

In [10]:
from config import db_password, db_name, db_server
db_string = f"postgresql://postgres:{db_password}@{db_server}/{db_name}"
engine = create_engine(db_string)
with engine.connect() as connection:
    covid_df = pd.read_sql("select * from covid_daily_info", db_string)
covid_df.head()  

,period_begin_month,period_begin_year,period_begin_month_year,covid_month_year_state,state,state_name_title_case,cases,deaths
0,1.0,2020.0,1-2020,1-2020CA,CA,California,0.0,0.0
1,2.0,2020.0,2-2020,2-2020CA,CA,California,34.0,0.0
2,3.0,2020.0,3-2020,3-2020CA,CA,California,6898.0,150.0
3,4.0,2020.0,4-2020,4-2020CA,CA,California,41985.0,1740.0
4,5.0,2020.0,5-2020,5-2020CA,CA,California,61666.0,2153.0


In [11]:
covid_df

,period_begin_month,period_begin_year,period_begin_month_year,covid_month_year_state,state,state_name_title_case,cases,deaths
0,1.0,2020.0,1-2020,1-2020CA,CA,California,0.0,0.0
1,2.0,2020.0,2-2020,2-2020CA,CA,California,34.0,0.0
2,3.0,2020.0,3-2020,3-2020CA,CA,California,6898.0,150.0
3,4.0,2020.0,4-2020,4-2020CA,CA,California,41985.0,1740.0
4,5.0,2020.0,5-2020,5-2020CA,CA,California,61666.0,2153.0
...,...,...,...,...,...,...,...,...
115,8.0,2021.0,8-2021,8-2021WA,WA,Washington,105088.0,451.0
116,9.0,2021.0,9-2021,9-2021WA,WA,Washington,112123.0,1152.0
117,10.0,2021.0,10-2021,10-2021WA,WA,Washington,82360.0,902.0
118,11.0,2021.0,11-2021,11-2021WA,WA,Washington,53169.0,675.0


### Converting month and year to int

In [12]:
covid_df['period_begin_month'] = covid_df['period_begin_month'].astype('int')
covid_df['period_begin_year'] = covid_df['period_begin_year'].astype('int')



### Checking for null values

In [13]:
covid_df.loc[covid_df['cases'].isnull()]

,period_begin_month,period_begin_year,period_begin_month_year,covid_month_year_state,state,state_name_title_case,cases,deaths
24,1,2020,1-2020,1-2020FL,FL,Florida,NaN,NaN
25,2,2020,2-2020,2-2020FL,FL,Florida,NaN,NaN
48,1,2020,1-2020,1-2020MN,MN,Minnesota,NaN,NaN
49,2,2020,2-2020,2-2020MN,MN,Minnesota,NaN,NaN
50,3,2020,3-2020,3-2020MN,MN,Minnesota,NaN,NaN
72,1,2020,1-2020,1-2020TX,TX,Texas,NaN,NaN
73,2,2020,2-2020,2-2020TX,TX,Texas,NaN,NaN


In [14]:
#Reference: https://dzone.com/articles/pandas-find-rows-where-columnfield-is-null
covid_df['cases'] = covid_df['cases'].fillna(0)
covid_df['deaths'] = covid_df['deaths'].fillna(0)

In [15]:
covid_df['cases'] = covid_df['cases'].astype('int')
covid_df['deaths'] = covid_df['deaths'].astype('int')

In [16]:
covid_df.head() 

,period_begin_month,period_begin_year,period_begin_month_year,covid_month_year_state,state,state_name_title_case,cases,deaths
0,1,2020,1-2020,1-2020CA,CA,California,0,0
1,2,2020,2-2020,2-2020CA,CA,California,34,0
2,3,2020,3-2020,3-2020CA,CA,California,6898,150
3,4,2020,4-2020,4-2020CA,CA,California,41985,1740
4,5,2020,5-2020,5-2020CA,CA,California,61666,2153


### Filter to see ZERO COVID Cases

In [17]:
covid_df.loc[covid_df['cases'] == 0]

,period_begin_month,period_begin_year,period_begin_month_year,covid_month_year_state,state,state_name_title_case,cases,deaths
0,1,2020,1-2020,1-2020CA,CA,California,0,0
24,1,2020,1-2020,1-2020FL,FL,Florida,0,0
25,2,2020,2-2020,2-2020FL,FL,Florida,0,0
48,1,2020,1-2020,1-2020MN,MN,Minnesota,0,0
49,2,2020,2-2020,2-2020MN,MN,Minnesota,0,0
50,3,2020,3-2020,3-2020MN,MN,Minnesota,0,0
72,1,2020,1-2020,1-2020TX,TX,Texas,0,0
73,2,2020,2-2020,2-2020TX,TX,Texas,0,0
97,2,2020,2-2020,2-2020WA,WA,Washington,0,1


### Filter to see ZERO COVID deaths

In [18]:
covid_df.loc[covid_df['deaths'] == 0]

,period_begin_month,period_begin_year,period_begin_month_year,covid_month_year_state,state,state_name_title_case,cases,deaths
0,1,2020,1-2020,1-2020CA,CA,California,0,0
1,2,2020,2-2020,2-2020CA,CA,California,34,0
24,1,2020,1-2020,1-2020FL,FL,Florida,0,0
25,2,2020,2-2020,2-2020FL,FL,Florida,0,0
48,1,2020,1-2020,1-2020MN,MN,Minnesota,0,0
49,2,2020,2-2020,2-2020MN,MN,Minnesota,0,0
50,3,2020,3-2020,3-2020MN,MN,Minnesota,0,0
72,1,2020,1-2020,1-2020TX,TX,Texas,0,0
73,2,2020,2-2020,2-2020TX,TX,Texas,0,0
96,1,2020,1-2020,1-2020WA,WA,Washington,1,0


# Importing Housing Data

In [21]:
from config import db_password, db_name, db_server
db_string = f"postgresql://postgres:{db_password}@{db_server}/{db_name}"
engine = create_engine(db_string)
with engine.connect() as connection:
    housing_df = pd.read_sql("select * from housing_data_by_state_by_month", db_string)
housing_df.head() 

,period_begin_month,period_begin_year,period_begin_month_year,state,state_code,housing_month_year_state,homes_sold,pending_sales,new_listings,inventory,months_of_supply,sold_above_list,price_drops
0,1,2020,1-2020,California,CA,1-2020CA,43730,37236,59735,117383,459,39.455874,28.325421
1,1,2020,1-2020,Florida,FL,1-2020FL,50212,45153,93867,249081,1203,20.296826,33.916778
2,1,2020,1-2020,Minnesota,MN,1-2020MN,7912,6372,9263,23367,1034,29.867686,24.239470
3,1,2020,1-2020,Texas,TX,1-2020TX,34846,32732,57370,147998,2213,34.811199,46.340096
4,1,2020,1-2020,Washington,WA,1-2020WA,12810,12124,14723,20983,373,22.953821,14.455445


### Checking for null values

In [22]:
housing_df.loc[housing_df['price_drops'].isnull()]

,period_begin_month,period_begin_year,period_begin_month_year,state,state_code,housing_month_year_state,homes_sold,pending_sales,new_listings,inventory,months_of_supply,sold_above_list,price_drops


In [23]:
housing_df.loc[housing_df['housing_month_year_state'].isnull()]

,period_begin_month,period_begin_year,period_begin_month_year,state,state_code,housing_month_year_state,homes_sold,pending_sales,new_listings,inventory,months_of_supply,sold_above_list,price_drops


In [24]:
housing_df.loc[housing_df['homes_sold'].isnull()]

,period_begin_month,period_begin_year,period_begin_month_year,state,state_code,housing_month_year_state,homes_sold,pending_sales,new_listings,inventory,months_of_supply,sold_above_list,price_drops


In [25]:
housing_df.loc[housing_df['pending_sales'].isnull()]

,period_begin_month,period_begin_year,period_begin_month_year,state,state_code,housing_month_year_state,homes_sold,pending_sales,new_listings,inventory,months_of_supply,sold_above_list,price_drops


In [26]:
housing_df.loc[housing_df['inventory'].isnull()]

,period_begin_month,period_begin_year,period_begin_month_year,state,state_code,housing_month_year_state,homes_sold,pending_sales,new_listings,inventory,months_of_supply,sold_above_list,price_drops


In [27]:
housing_df.loc[housing_df['months_of_supply'].isnull()]

,period_begin_month,period_begin_year,period_begin_month_year,state,state_code,housing_month_year_state,homes_sold,pending_sales,new_listings,inventory,months_of_supply,sold_above_list,price_drops


In [28]:
housing_df.loc[housing_df['sold_above_list'].isnull()]

,period_begin_month,period_begin_year,period_begin_month_year,state,state_code,housing_month_year_state,homes_sold,pending_sales,new_listings,inventory,months_of_supply,sold_above_list,price_drops


In [29]:
housing_df.loc[housing_df['price_drops'].isnull()]

,period_begin_month,period_begin_year,period_begin_month_year,state,state_code,housing_month_year_state,homes_sold,pending_sales,new_listings,inventory,months_of_supply,sold_above_list,price_drops


### Checking for null values using 'isna'

In [30]:
housing_df.isna().sum()

period_begin_month          0
period_begin_year           0
period_begin_month_year     0
state                       0
state_code                  0
housing_month_year_state    0
homes_sold                  0
pending_sales               0
new_listings                0
inventory                   0
months_of_supply            0
sold_above_list             0
price_drops                 0
dtype: int64

In [31]:
housing_df.dtypes

period_begin_month            int64
period_begin_year             int64
period_begin_month_year      object
state                        object
state_code                   object
housing_month_year_state     object
homes_sold                    int64
pending_sales                 int64
new_listings                  int64
inventory                     int64
months_of_supply              int64
sold_above_list             float64
price_drops                 float64
dtype: object

In [32]:
covid_df.dtypes

period_begin_month          int32
period_begin_year           int32
period_begin_month_year    object
covid_month_year_state     object
state                      object
state_name_title_case      object
cases                       int32
deaths                      int32
dtype: object

### Merging dataframes

In [38]:
#Reference: https://stackoverflow.com/questions/25888207/pandas-join-dataframes-on-field-with-different-names
#Reference: https://stackoverflow.com/questions/19125091/pandas-merge-how-to-avoid-duplicating-columns

covid_housing_df = pd.merge(housing_df,covid_df,how='left', left_on = ['housing_month_year_state'], right_on=['covid_month_year_state'], suffixes=('','_y'))
covid_housing_df.drop(covid_housing_df.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)

In [39]:
covid_housing_df.head()


,period_begin_month,period_begin_year,period_begin_month_year,state,state_code,housing_month_year_state,homes_sold,pending_sales,new_listings,inventory,months_of_supply,sold_above_list,price_drops,covid_month_year_state,state_name_title_case,cases,deaths
0,1,2020,1-2020,California,CA,1-2020CA,43730,37236,59735,117383,459,39.455874,28.325421,1-2020CA,California,0,0
1,1,2020,1-2020,Florida,FL,1-2020FL,50212,45153,93867,249081,1203,20.296826,33.916778,1-2020FL,Florida,0,0
2,1,2020,1-2020,Minnesota,MN,1-2020MN,7912,6372,9263,23367,1034,29.867686,24.239470,1-2020MN,Minnesota,0,0
3,1,2020,1-2020,Texas,TX,1-2020TX,34846,32732,57370,147998,2213,34.811199,46.340096,1-2020TX,Texas,0,0
4,1,2020,1-2020,Washington,WA,1-2020WA,12810,12124,14723,20983,373,22.953821,14.455445,1-2020WA,Washington,1,0


In [40]:
covid_housing_df.head()

,period_begin_month,period_begin_year,period_begin_month_year,state,state_code,housing_month_year_state,homes_sold,pending_sales,new_listings,inventory,months_of_supply,sold_above_list,price_drops,covid_month_year_state,state_name_title_case,cases,deaths
0,1,2020,1-2020,California,CA,1-2020CA,43730,37236,59735,117383,459,39.455874,28.325421,1-2020CA,California,0,0
1,1,2020,1-2020,Florida,FL,1-2020FL,50212,45153,93867,249081,1203,20.296826,33.916778,1-2020FL,Florida,0,0
2,1,2020,1-2020,Minnesota,MN,1-2020MN,7912,6372,9263,23367,1034,29.867686,24.239470,1-2020MN,Minnesota,0,0
3,1,2020,1-2020,Texas,TX,1-2020TX,34846,32732,57370,147998,2213,34.811199,46.340096,1-2020TX,Texas,0,0
4,1,2020,1-2020,Washington,WA,1-2020WA,12810,12124,14723,20983,373,22.953821,14.455445,1-2020WA,Washington,1,0


### Splitting

In [ ]:
covid_housing_df_encoded = pd.get_dummies(covid_housing_df, columns=["covid_month_year_state", ""])
covid_housing_df_encoded.head()


In [ ]:
X = covid_housing_df.drop('homes_sold', axis=1)
y = covid_housing_df['homes_sold'] 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)

In [ ]:
scaler  = StandardScaler()

In [ ]:
scaler.fit(X_train)